[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adugnag/deSpeckNet-TF-GEE/blob/main/notebooks/Prepare_data.ipynb)

# Setup software libraries



In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import tensorflow as tf
import numpy as np

#tf.enable_eager_execution()
print(tf.__version__)

# Folium setup.
import folium
print(folium.__version__)

In [ ]:
#@title Helper functions for data prep
###########################################
# CONVERT LINEAR TO DB
###########################################
def lin_to_db(image):
    """
    Convert backscatter from linear to dB.
    Parameters
    ----------
    image : ee.Image
        Image to convert 
    Returns
    -------
    ee.Image
        output image
    """
    bandNames = image.bandNames().remove('angle')
    db = ee.Image.constant(10).multiply(image.select(bandNames).log10()).rename(bandNames)
    return image.addBands(db, None, True)

###########################################
# PREPARE
###########################################

def s1_prep(params):
    """
    Prepares a collection of S1 images using a dictionary of parameters. 

    """
    
    POLARIZATION = params['POLARIZATION']
    FORMAT = params['FORMAT']
    START_DATE = params['START_DATE']
    STOP_DATE = params['STOP_DATE']
    ORBIT = params['ORBIT']
    RELATIVE_ORBIT_NUMBER = params['RELATIVE_ORBIT_NUMBER']
    ROI = params['ROI']
    CLIP_TO_ROI = params['CLIP_TO_ROI']
    
    if POLARIZATION is None: POLARIZATION = 'VVVH'
    if FORMAT is None: FORMAT = 'DB' 
    if ORBIT is None: ORBIT = 'DESCENDING' 
    
    
    pol_required = ['VV', 'VH', 'VVVH']
    if (POLARIZATION not in pol_required):
        raise ValueError("ERROR!!! Parameter POLARIZATION not correctly defined")

    
    orbit_required = ['ASCENDING', 'DESCENDING', 'BOTH']
    if (ORBIT not in orbit_required):
        raise ValueError("ERROR!!! Parameter ORBIT not correctly defined")


    format_required = ['LINEAR', 'DB']
    if (FORMAT not in format_required):
        raise ValueError("ERROR!!! FORMAT not correctly defined")
        
    
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
                .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
                .filter(ee.Filter.eq('resolution_meters', 10)) \
                .filter(ee.Filter.eq('platform_number', 'A')) \
                .filterDate(START_DATE, STOP_DATE) \
                .filterBounds(ROI)
    

        # select orbit
    if (ORBIT != 'BOTH'):
      s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', ORBIT))

    if (RELATIVE_ORBIT_NUMBER != 'ANY'): 
      s1 =  s1.filter(ee.Filter.eq('relativeOrbitNumber_start', RELATIVE_ORBIT_NUMBER)) 
      
    
    if (POLARIZATION == 'VV'):
      s1 = s1.select(['VV','angle'])
    elif (POLARIZATION == 'VH'):
      s1 = s1.select(['VH','angle'])
    elif (POLARIZATION == 'VVVH'):
      s1 = s1.select(['VV','VH','angle'])  
    
    # clip image to roi
    if (CLIP_TO_ROI):
        s1 = s1.map(lambda image: image.clip(ROI))
    
    if (FORMAT == 'DB'):
        s1 = s1.map(lin_to_db)
        
        
    return s1

###########################################
# EXPORT TRAINING DATA
###########################################

def exportDataset(params):
  trainingPolysList = train_poly.toList(train_poly.size())
  evalPolysList = val_poly.toList(val_poly.size())

  # These numbers determined experimentally.
  n = 250 # Number of shards in each polygon.
  N = 2500 # Total sample size in each polygon.

  # Export all the training data (in many pieces), with one task per geometry.
  for g in range(train_poly.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(trainingPolysList.get(g)).geometry(), 
        scale = 10, 
        numPixels = N / n, # Size of the shard.
        seed = i,
        tileScale = 16
    )
    geomSample = geomSample.merge(sample)
  
    desc = params['TRAINING_BASE'] + '_g' + str(g)
    if params['EXPORT'] == 'GCS':
        task = ee.batch.Export.table.toCloudStorage(
            collection = geomSample,
            description = desc, 
            bucket = params['BUCKET'], 
            fileNamePrefix = params['FOLDER'] + '/' + desc,
            fileFormat = 'TFRecord',
            selectors = FEATURES
            )
    else:
        task = ee.batch.Export.table.toDrive(
            collection = geomSample,
            description = desc,  
            fileNamePrefix = params['FOLDER'] + '/' + desc,
            fileFormat = 'TFRecord',
            selectors = FEATURES
            )
    task.start()

  # Export all the evaluation data.
  for g in range(val_poly.size().getInfo()):
    geomSample = ee.FeatureCollection([])
    for i in range(n):
      sample = arrays.sample(
        region = ee.Feature(evalPolysList.get(g)).geometry(), 
        scale = 10, 
        numPixels = N / n,
        seed = i,
        tileScale = 16
      )
      geomSample = geomSample.merge(sample)
  
    desc = params['EVAL_BASE'] + '_g' + str(g)
    if params['EXPORT'] == 'GCS':
        task = ee.batch.Export.table.toCloudStorage(
            collection = geomSample,
            description = desc, 
            bucket = params['BUCKET'], 
            fileNamePrefix = params['FOLDER'] + '/' + desc,
            fileFormat = 'TFRecord',
            selectors = FEATURES
            )
    else:
        task = ee.batch.Export.table.toDrive(
            collection = geomSample,
            description = desc, 
            fileNamePrefix = params['FOLDER'] + '/' + desc,
            fileFormat = 'TFRecord',
            selectors = FEATURES
            )
    task.start()


# Parameters



In [ ]:


#ROI
geometry = ee.Geometry.Polygon(
        [[[103.08000490033993, -2.8225068747308946],
          [103.08000490033993, -2.9521181019620673],
         [103.29217836225399, -2.9521181019620673],
         [103.29217836225399, -2.8225068747308946]]])

#parameters
params = {  'START_DATE': '2021-01-01', 
            'STOP_DATE': '2021-12-31',        
            'ORBIT': 'DESCENDING',
            'RELATIVE_ORBIT_NUMBER':18, 
            'POLARIZATION': 'VVVH',
            'ROI':    geometry,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': False,
          # GCS bucket
            'EXPORT': 'GCS',
            'BUCKET' : 'senalerts_dl3',
            'DRIVE' : '/content/drive',
            'FOLDER' : 'deSpeckNet',
            'TRAINING_BASE' : 'training_deSpeckNet_DUAL_Median_mask',
            'EVAL_BASE' : 'eval_deSpeckNet_DUAL_median_mask',
            'MODE' : 'training',
            'KERNEL_SIZE' : 40,
            'KERNEL_SHAPE' : [40, 40],
            }

#process Sentinel 1 image collection
s1_processed = s1_prep(params)
bandNames = s1_processed.first().bandNames().remove('angle')
print(bandNames.getInfo())
s1_processed = s1_processed.select(bandNames)
print('Number of images in the collection: ', s1_processed.size().getInfo())

#n = s1_processed.size().getInfo();
#colList = s1_processed.toList(n);

image = s1_processed.first()
label =s1_processed.reduce(ee.Reducer.median())
stddev = s1_processed.reduce(ee.Reducer.stdDev())
#Mask out pixels with high stdDev. Threshold is higher as the data is in dB.
#maskBand = ['VV_mask', 'VH_mask']
maskBand = bandNames.map(lambda bandName: ee.String(bandName).cat('_mask'))
mask = stddev.lte(2.0).rename(maskBand)


In [ ]:
print(mask.bandNames().getInfo())

#Features

In [ ]:

# Specify inputs (Sentinel-1 bands) to the model and the response variable.
BANDS = bandNames.getInfo()
#number of selected images
print('List of band names in input: ', BANDS)

RESPONSE_TR = label.bandNames().getInfo()
RESPONSE_TU = bandNames

MASK = mask.bandNames().getInfo()

if params['MODE'] == 'training':
  FEATURES = BANDS + RESPONSE_TR + MASK
else:
  FEATURES = BANDS + RESPONSE_TU

print('List of feature names in input: ', FEATURES)



#Sampling regions

In [ ]:
#Sampling polygons. If using new areas please adjust these polygons

train1 = ee.Geometry.Polygon(
        [[[103.23827668989107, -2.82473762348129],
          [103.23827668989107, -2.8730863008895193],
          [103.28908845746919, -2.8730863008895193],
          [103.28908845746919, -2.82473762348129]]])
          
train2 = ee.Geometry.Polygon(
        [[[103.08275148237153, -2.826109245073874],
          [103.08275148237153, -2.891945160858568],
          [103.1740753349106, -2.891945160858568],
          [103.1740753349106, -2.826109245073874]]])
          
train3 = ee.Geometry.Polygon(
        [[[103.12017366254732, -2.887830527175443],
          [103.12017366254732, -2.94954845545005],
          [103.245829790477, -2.94954845545005],
          [103.245829790477, -2.887830527175443]]])

train4 = geometry = ee.Geometry.Polygon(
        [[[103.15527841413423, -2.8621137297832173],
          [103.15527841413423, -2.9077177848426454],
          [103.21124002302095, -2.9077177848426454],
          [103.21124002302095, -2.8621137297832173]]])

train5 = ee.Geometry.Polygon(
        [[[103.10892984235689, -2.8934881446424914],
          [103.10892984235689, -2.9373765773812446],
          [103.1755344566147, -2.9373765773812446],
          [103.1755344566147, -2.8934881446424914]]])

train6 =  ee.Geometry.Polygon(
        [[[103.17373201215669, -2.8252519817684183],
          [103.17373201215669, -2.9037746494754204],
          [103.26368257368013, -2.9037746494754204],
          [103.26368257368013, -2.8252519817684183]]])
train7 =  ee.Geometry.Polygon(
        [[[103.081034868602, -2.904460413137298],
          [103.081034868602, -2.9500627572273728],
          [103.13424989545747, -2.9500627572273728],
          [103.13424989545747, -2.904460413137298]]])
          
val1 = ee.Geometry.Polygon(
        [[[103.24273988569185, -2.8478835202194768],
          [103.24273988569185, -2.9483484170446874],
          [103.29046174848482, -2.9483484170446874],
          [103.29046174848482, -2.8478835202194768]]])
          
val2 = ee.Geometry.Polygon(
        [[[103.08240815961763, -2.8566274048086933],
          [103.08240815961763, -2.8988028504820584],
          [103.1301300224106, -2.8988028504820584],
          [103.1301300224106, -2.8566274048086933]]])

val3 =     ee.Geometry.Polygon(
        [[[103.15527841413423, -2.8609135984375555],
          [103.15527841413423, -2.8972598739369757],
          [103.21295663679048, -2.8972598739369757],
          [103.21295663679048, -2.8609135984375555]]])

train_poly = ee.FeatureCollection([ee.Feature(train1),
                                   ee.Feature(train2),
                                   ee.Feature(train3),
                                   ee.Feature(train4),
                                   ee.Feature(train5),
                                   ee.Feature(train6),
                                   ee.Feature(train7)])

val_poly =  ee.FeatureCollection([ee.Feature(val1),
                                   ee.Feature(val2),
                                  ee.Feature(val3)])

polyImage = ee.Image(0).byte().paint(train_poly, 1).paint(val_poly, 2)
polyImage = polyImage.updateMask(polyImage)


#Visualize

In [ ]:
# Use folium to visualize the imagery.#
mask_mapid = mask.getMapId({'bands':MASK[0], 'min': 0, 'max':1})
s1_mapid = image.getMapId({'bands': BANDS[0], 'min': -20, 'max':0})
label_mapid = label.getMapId({'bands': RESPONSE_TR[0],'min': -20, 'max': 0})
poly_mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-2.6145179357243027, 103.46795961225435], zoom_start=14)

folium.TileLayer(
    tiles=mask_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Mask',
  ).add_to(map)
folium.TileLayer(
    tiles=label_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Label',
  ).add_to(map)
folium.TileLayer(
    tiles=s1_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Image',
  ).add_to(map)
folium.TileLayer(
    tiles=poly_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Train and eval polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

#Export

In [ ]:
#create a feature stack
if params['MODE'] == 'training':
  featureStack = ee.Image.cat([
                image.select(BANDS),
                label.select(RESPONSE_TR),
                mask.select(MASK)
                ])
else:
  featureStack = ee.Image.cat([
                image.select(BANDS),
                label.select(RESPONSE_TU)
                ])

list = ee.List.repeat(1, params['KERNEL_SIZE'])
lists = ee.List.repeat(list, params['KERNEL_SIZE'])
kernel = ee.Kernel.fixed(params['KERNEL_SIZE'], params['KERNEL_SIZE'], lists)

arrays = featureStack.neighborhoodToArray(kernel)

#export dataset
exportDataset(params)